<div>
<div style="text-align:center; display:block; float:left; padding:80px;"><img width="200px"  src="https://kaggle2.blob.core.windows.net/competitions/kaggle/4651/logos/front_page.png"/><span style="color:red;">**New User Booking**</span></div>
<div style="">
**Objective:** In this recruiting competition, Airbnb challenges you to predict in which country a new user will make his or her first booking.  
  
** Description: ** In this challenge, you are given a list of users along with their demographics, web session records, and some summary statistics. You are asked to predict which country a new user's first booking destination will be. All the users in this dataset are from the USA.
</div>
<img src="https://kaggle2.blob.core.windows.net/competitions/kaggle/4651/media/airbnb_banner.png" />

## Import Packages

In [53]:
using DataFrames
using MLBase
using XGBoost

## Load Data

In [ ]:
demographic_data

In [74]:
train = readtable("data/train_v1.tsv", separator='\t')
test  = readtable("data/test_v1.tsv", separator='\t')
full  = readtable("data/full_v1.tsv", separator='\t');

## Set Features and Output

In [54]:
label    = :country_destination
labels   = Set(train[label])
features = setdiff(names(test), [:id]);

original_labels = keys(labelmap(readtable("data/train_users_2.csv.gz")[label]));

## Prepare Instances

In [12]:
function split_train_val(df; train_size=.85, random_state=1)
    
    srand(random_state)
    
    nrows, ntraining_rows = size(df, 1), round(Int, size(df, 1) * train_size)
    indexes               = shuffle(collect(1:nrows))
    train                 = df[indexes[1:ntraining_rows], :]
    validation            = df[indexes[ntraining_rows+1:end], :]
    
    return train, validation
end

split_train_val (generic function with 1 method)

In [75]:
X_train, X_val = split_train_val(train, train_size=.85, random_state=1)

train_x = Array{Float64,2}(X_train[:, features])
train_y = Array{Float64,1}(X_train[label])
val_x   = Array{Float64,2}(X_val[:, features])
val_y   = Array{Float64,1}(X_val[label])
test_x  = Array{Float64,2}(test[:, features]);

LoadError: LoadError: KeyError: YHat not found
while loading In[75], in expression starting on line 3

In [76]:
dtrain  = DMatrix(train_x, label=train_y)
dval    = DMatrix(val_x, label=val_y);

## Train

In [ ]:
num_rounds = 100
watchlist  = [(dtrain, "train"), (dval, "eval")]
metrics    = ["merror", "mlogloss"]
params     = Dict({"objective" => "multi:softmax",
                   "booster"   => "gbtree",
                   "eta"       => 0.1, 
                   "max_depth" => 12,
                   "subsample" => 0.85
                  })

println("Training Base Model...")
tic()
model      = XGBoost.xgboost(dtrain, num_rounds, param=params, metrics=metrics,
                             num_class=length(labels)+1, watchlist=watchlist)
toc()


Use "Dict{Any,Any}(a=>b, ...)" instead.


Training Base Model...


[1]	train-merror:0.361081	train-mlogloss:2.275779	eval-merror:0.367887	eval-mlogloss:2.280345
[2]	train-merror:0.357961	train-mlogloss:2.084641	eval-merror:0.367543	eval-mlogloss:2.094408
[3]	train-merror:0.355685	train-mlogloss:1.940276	eval-merror:0.367606	eval-mlogloss:1.954366
[4]	train-merror:0.353987	train-mlogloss:1.824317	eval-merror:0.368012	eval-mlogloss:1.842318
[5]	train-merror:0.352593	train-mlogloss:1.728214	eval-merror:0.367574	eval-mlogloss:1.750234
[6]	train-merror:0.351270	train-mlogloss:1.646612	eval-merror:0.367543	eval-mlogloss:1.672501
[7]	train-merror:0.350284	train-mlogloss:1.576480	eval-merror:0.366981	eval-mlogloss:1.606201
[8]	train-merror:0.348531	train-mlogloss:1.514960	eval-merror:0.367325	eval-mlogloss:1.548508
[9]	train-merror:0.347131	train-mlogloss:1.460771	eval-merror:0.367137	eval-mlogloss:1.498255
[10]	train-merror:0.345979	train-mlogloss:1.412995	eval-merror:0.367200	eval-mlogloss:1.453982
[11]	train-merror:0.345329	train-mlogloss:1.370483	eval-mer

## Predict

In [62]:
test[:YHat] = convert(Array{Int32,1}, XGBoost.predict(model, test_x));

## Generate Submission File

In [48]:
function prepare_dataframe_submission(df)
    
    ids           = df[:id]
    yhats         = [ original_labels[df[i, :YHat]] for i=1:size(df, 1) ]
    submission_df = DataFrame(id=ids, country=yhats)
    
    return submission_df
end

prepare_dataframe_submission (generic function with 1 method)

In [63]:
submission_df = prepare_dataframe_submission(test);

.927184815 seconds


In [64]:
writetable("data/submissions/submission_v2_xgboost_msoftmax_gbtree_eta7_md5_ss85_nr2000.csv", submission_df);

## Kagle Scores from Submited Predictions

Best Score: <span style="color:blue;">0.70174</span> [v1]  
- v2 **XGBoost** (MSoftMax GBTree Eta.7 MD 5 SS.85 NR2000): ** 0.65697** tme.181 tmll.499 eme.403 emll1.477
- v1 **XGBoost** (MSoftMax GBTree Eta.7 MD 5 SS.85 NR100): **0.70174** tme.349 tmll.948 eme.367 emll1.077